In [4]:
pip install transformers ffmpeg pytelegrambotapi langchain langchain-openai pydub protobuf langchain-huggingface ipywidgets langchain-ollama langchain-community

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from transformers import pipeline

whisper = pipeline("automatic-speech-recognition", 
                   "openai/whisper-large-v3-turbo", 
                   torch_dtype=torch.float16, 
                   device="cuda:0",
                   return_timestamps=True)

In [6]:
# from transformers import AutoTokenizer, LlamaForCausalLM

# model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

# def llama(prompt):
#     inputs = tokenizer(prompt, return_tensors="pt")
#     generate_ids = model.generate(inputs.input_ids, max_length=30)
    
#     return tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [7]:
# from langchain_huggingface import HuggingFacePipeline

# llm = HuggingFacePipeline.from_model_id(
#     model_id="microsoft/Phi-3-mini-4k-instruct-gguf",
#     task="text-generation",
#     device=0,
#     pipeline_kwargs={
#         "temperature": 0.1,
#         'max_length': 1000
#     },
# )

In [2]:
from langchain_community.llms import Ollama

llm = Ollama(
    base_url='http://172.19.0.2:11434',
    model='openhermes:7b-mistral-v2.5-q8_0'
)

llm.invoke('how u doing')

/tmp/ipykernel_454/3362342825.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


"I'm just a text-based AI, but I'm here to help and assist you in any way I can! How can I help you today?"

In [9]:
#ollama.invoke('Как дела братан?')

In [10]:
def transcribe_whisper(mp3_path):
    transcription = whisper(mp3_path)

    return transcription["text"]

In [3]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI


def summarize_with_llm(text):
    messages = [
        SystemMessage(content="Суммаризуй сообщение пользователя. Выдели основные мысли. Ответ оформи в формате markdown."),
        HumanMessage(content=text)
    ]

    return llm.invoke(messages)

In [37]:
def to_markdown(t):
    if '```' not in t:
        return '```markdown\n' + t + '\n```'
    else: 
        return t

def summarize_mp3(path):
    transcribtion = transcribe_whisper(path)

    summarization = summarize_with_llm(transcribtion)

    return summarization

def summarize_aslan(text):
    summ = summarize_with_llm(text)
    
    messages = [
        SystemMessage(content="Перефразируй сообщение. При формировании ответа ты обязательно должен использовать все 3 фразы:\n1. Засади ей\n2. Я взгрелся\n3. Сейчас я засажу смачно Тёме"),
        HumanMessage(content=summ)
    ]

    return llm.invoke(messages)

In [48]:
import json

with open('bot_info.json', 'w') as f:
    json.dump({'users': [], 'admins': [], 'aslan_mode': []}, f)

In [ ]:
import telebot
import traceback
import os
import json
from pprint import pprint
from pathlib import Path
import shutil
from pydub import AudioSegment

mp3_file = "record.mp3"


with open('bot_info.json', 'r') as f:
    global users
    global admins
    global aslan_mode
    
    d = json.load(f)
    users = d.get('users') or ['tcarroflan']
    admins = d.get('admins') or ['tcarroflan']
    aslan_mode = d.get('aslan_mode') or []

    
telebot.apihelper.API_URL = 'http://localhost:8081/bot{0}/{1}'
bot = telebot.TeleBot(token='***')
botfile_path = Path('../var/lib/telegram-bot-api/***')

def with_user_validation(func):
    def wrapper(message):
        if message.from_user.username in users:
            return func(message)
        else:
            return None
    return wrapper

def with_admin_validation(func):
    def wrapper(message):
        if message.from_user.username in admins:
            return func(message)
        else:
            return None
    return wrapper

def user_in_aslan_mode(user):
    return user in aslan_mode

def update_bot_info():
    os.remove('bot_info.json')
    
    with open('bot_info.json', 'w') as f:
        json.dump({'users': users, 'admins': admins, 'aslan_mode': aslan_mode}, f)

def bot_command(func):
    def wrapper(message):
        update_bot_info()
        
        try:
            cmd, user = message.text.strip().split()
            bot.reply_to(message, func(user))
        except:
            bot.reply_to(message, f'Не могу выполнить операцию, команда должна быть в формате "/команда ник"')

    return wrapper


@bot.message_handler(commands=['add_user'])
@with_admin_validation
@bot_command
def add_user(name):
    if name not in users:
        users.append(name)
        return f'Пользователь {name} добавлен'
    else:
        return f'Пользователь {name} уже в списке'


@bot.message_handler(commands=['drop_user'])
@with_admin_validation
@bot_command
def drop_user(name):
    global users
    
    if name in users:
        users.remove(name)
        return f'Пользователь {name} удален'
    else:
        return f'Пользователь {name} итак не в списке'


@bot.message_handler(commands=['add_admin'])
@with_admin_validation
@bot_command
def add_admin(name):
    if name not in admins:
        admins.append(name)
        
        if name not in users:
            users.append(name)
    
        return f'Админ {name} добавлен'
    else:
        return f'Админ {name} уже в списке'


@bot.message_handler(commands=['drop_admin'])
@with_admin_validation
@bot_command
def drop_admin(name):
    global admins
    
    if name in admins:
        admins.remove(name)
        return f'Админ {name} удален'
    else:
        return f'Админ {name} итак не в списке'


@bot.message_handler(commands=['add_aslan'])
@with_admin_validation
@bot_command
def add_aslan(name):
    if name not in aslan_mode:
        aslan_mode.append(name)
        
        if name not in users:
            users.append(name)
    
        return f'Аслано-любитель {name} добавлен'
    else:
        return f'Аслано-любитель {name} уже в списке'


@bot.message_handler(commands=['drop_aslan'])
@with_admin_validation
@bot_command
def drop_aslan(name):
    global aslan_mode
    
    if name in aslan_mode:
        aslan_mode.remove(name)
        return f'Аслано-любитель {name} удален'
    else:
        return f'Аслано-любитель {name} итак не в списке'


@bot.message_handler(commands=['get_users'])
@with_user_validation
def bot_get_users(message):
    bot.reply_to(message, f"Список пользователей:\n{users}")


@bot.message_handler(commands=['get_admins'])
@with_admin_validation
def bot_get_admins(message):
    bot.reply_to(message, f"Список адмиинов:\n{admins}")


@bot.message_handler(commands=['get_aslans'])
@with_admin_validation
def bot_get_aslans(message):
    bot.reply_to(message, f"Список Аслано-любителей:\n{aslan_mode}")


@bot.message_handler(commands=['start', 'help'])
def handle_help(message):
    text = (
        "Привет, я бот - суммаризатор медиа файлов!\n\n"
        "Пришли мне видео, аудио, голосовое или кружочек, а я отправлю тебе краткую сводку из него.\n\n"
        "Или можешь просто пообщаться со мной)"
    )
    bot.send_message(message.chat.id, text)


@bot.message_handler(content_types=['audio', 'video', 'voice', 'video_note'])
@with_user_validation
def bot_get_audio(message):
    try:
        if message.audio:
            file_info = bot.get_file(message.audio.file_id)

        if message.video:
            file_info = bot.get_file(message.video.file_id)

        if message.voice:
            file_info = bot.get_file(message.voice.file_id)

        if message.video_note:
            file_info = bot.get_file(message.video_note.file_id)
            
        file_path = botfile_path / file_info.file_path
            
        try:
            AudioSegment.from_file(file_path).export(mp3_file, format="mp3")
            bot.reply_to(message, 'Ушел работать!')
        except:
            bot.reply_to(message, 'Сори, c таким форматом не работаю (')
            return None

        transcribtion = transcribe_whisper(mp3_file)

        if not user_in_aslan_mode(message.from_user.username):
            summarization = summarize_aslan(transcribtion)
        else:
            summarization = summarize_with_llm(transcribtion)
        
        #bot.send_message(message.from_user.id, 'Съел текст из видео')
        #md = to_markdown(summarization)
        
        bot.reply_to(message, 'Краткое содержание:\n\n' + summarization)#, parse_mode='MarkdownV2')

        os.remove('record.mp3')
        os.remove(file_path)
        
    except:
        bot.reply_to(message, traceback.format_exc())


@bot.message_handler(content_types=['text'])
@with_user_validation
def bot_get_text(message):
    try:
        if not user_in_aslan_mode(message.from_user.username):
            bot.reply_to(message, llm.invoke([HumanMessage(content=message.text)]))

        else:
            messages = [
                SystemMessage(content="Ответь пользователю. При формировании ответа ты обязательно должен использовать все 3 фразы:\n1. Засади ей\n2. Я взгрелся\n3. Сейчас я засажу смачно Тёме"),
                HumanMessage(content=message.text)
            ]

            bot.reply_to(message, llm.invoke(messages))
    except:
        bot.reply_to(message, traceback.format_exc())


bot.polling(none_stop=True, interval=0)

In [18]:
# Поднять кастомную LLM можно тут же в тетрадке +
# Сделать работу в группе
# Сделать работу с норм типами файлов +
# Сделать редактирование вайтилста +